<a href="https://colab.research.google.com/github/mvsamudrala/PLAIG/blob/main/PLAIG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import HTML

In [ ]:
%%capture
start = time.time()
normalization_statistics_file = "combined_set_normalization_statistics.pkl"
with open(normalization_statistics_file, 'rb') as file:
    normalization_statistics = pickle.load(file)
# graph_data_file = "DUDE_Z_norm_3.pkl"
# with open(graph_data_file, 'rb') as file:
#     dataset = pickle.load(file)
# print(len(dataset))
warnings.filterwarnings('ignore')
cannot_read_mols = []
count = 0
distance_cutoff = 3
pre_dataset = []
all_ligand_features = []
all_pocket_features = []
# all_ligand_features_key = {}
# all_pocket_features_key = {}
# dataset = []
no_nodes_count = 0
for protein, ligands in dudez_dataset.items():
    random.shuffle(ligands)
    count = 0
    protein_name = os.path.splitext(os.path.basename(protein))[0]
    print(protein_name)
    protein_pocket_path = protein
    print(protein_pocket_path)
    protein_pocket_pdbqt_path = f"/Users/mvsamudrala/BindingAffinityGNN/PDBQT_DUDE_Z_only/{protein_name}/{protein_name}.pdbqt"
    print(protein_pocket_pdbqt_path)
    # protein_path = "/Users/mvsamudrala/BindingAffinityGNN/refined-set/1a1e/1a1e_protein.pdb"
    # pdbqt_output_dir = "/Users/mvsamudrala/BindingAffinityGNN/PDBQT_refined"
    # path_parts = protein_pocket_path.split("/")
    # pdb_code = path_parts[-2]
    for ligand in ligands:
        print(count)
        ligand_name = os.path.splitext(os.path.basename(ligand))[0]
        print(ligand_name)
        ligand_path = ligand
        print(ligand_path)
        if "xtal" in ligand_name:
            ligand_pdbqt_path = f"/Users/mvsamudrala/BindingAffinityGNN/PDBQT_DUDE_Z_only/{protein_name}/{ligand_name}.pdbqt"
        elif "ZINC" in ligand_name:
            # ligand_pdbqt_path = f"/Users/mvsamudrala/BindingAffinityGNN/PDBQT_DUDE_Z_only/{protein_name}/decoys/{ligand_name}.pdbqt"
            continue
        else:
            if protein_name == "AMPC" or protein_name == "DRD4" or protein_name == "MT1":
                ligand_pdbqt_path = f"/Users/mvsamudrala/BindingAffinityGNN/PDBQT_DUDE_Z_only/{protein_name}/ligands/{ligand_name}.pdbqt"
            else:
                continue
        print(ligand_pdbqt_path)
        print()
        try:
            graph = GNN_representationv5.pl_complex_to_graph(protein_pocket_path, ligand_path, protein_pocket_pdbqt_path, ligand_pdbqt_path, distance_cutoff)
            # for edge in graph.edges(data=True):
            #     print(edge)
            if len(graph.nodes) == 0:
                no_nodes_count += 1
                print(f"Graph has no nodes, #{no_nodes_count}")
                continue
            pre_dataset.append((graph, (protein_name, ligand_name)))
            # all_ligand_features_key[code] = graph.graph["ligand_attr"]
            # all_pocket_features_key[code] = graph.graph["pocket_attr"]
            print(graph.graph["ligand_attr"])
            all_ligand_features.append(graph.graph["ligand_attr"])
            print(graph.graph["pocket_attr"])
            all_pocket_features.append(graph.graph["pocket_attr"])
            # in_channels = graph_data.num_node_features
            # edge_channels = graph_data.edge_features.shape[1]
            # hidden_channels = 32
            # out_channels = 1
            # num_graph_features = len(graph_data.graph_features)
            #
            # model = GCN(in_channels, edge_channels, hidden_channels, out_channels, num_graph_features)
            #
            # print(model)

        except Exception as e:
            cannot_read_mols.append((protein_name, ligand_name))
            print(f"Cannot read {protein_name}, {ligand_name} file: {e}")
        # if count == 1000:
        #     break
        count += 1
print()
all_ligand_features_normalized, all_pocket_features_normalized = normalize_graph_features(all_ligand_features, all_pocket_features, normalization_statistics)
for index, (graph, (protein_name, ligand_name)) in enumerate(pre_dataset):
    graph.graph["ligand_attr"] = all_ligand_features_normalized[index]
    print(graph.graph["ligand_attr"])
    graph.graph["pocket_attr"] = all_pocket_features_normalized[index]
    print(graph.graph["pocket_attr"])
    graph_data = prepare_data(graph, f"{protein_name}, {ligand_name}")
    # count = 0
    # for edge in graph.edges():
    #     print(count)
    #     print(edge)
    #     count += 1
    print(f"Data object: {graph_data}")
    print(f'Number of nodes: {graph_data.num_nodes}')
    print(f'Number of edges: {graph_data.num_edges}')
    print(f'Average node degree: {graph_data.num_edges / graph_data.num_nodes:.2f}')
    print(f'Has isolated nodes: {graph_data.has_isolated_nodes()}')
    print(f'Has self-loops: {graph_data.has_self_loops()}')
    print(f'Is undirected: {graph_data.is_undirected()}')
    print()
    dataset.append(graph_data)
print(len(dataset))
graph_data_file = "DUDE_Z_norm_3_pt2.pkl"
with open(graph_data_file, 'wb') as file:
    pickle.dump(dataset, file)

Input your .PDB and .PDBQT protein and ligand files for preprocessing.